In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc

from datetime import datetime
import requests
import json
from pandas import json_normalize

from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

In [2]:
boston = load_boston()

C:\Users\gisic\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (

In [3]:
today = datetime.today().strftime('%Y%m%d')
year_list =[2020, 2021, 2022]
df_ho =pd.DataFrame([])
for i in year_list:
    today_year = i

    key = 'UhnjyvmWEq5SH4yCqPylCPQSp4NdA524up%2FM1CrsUf3143Vmj3Rwx7PGHxq5YZY4mFHNHQ%2F0ixGwapRFApPsaw%3D%3D'
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear=' + str(today_year) + '&ServiceKey=' + str(key)
    response = requests.get(url)
    if response.status_code == 200:
        json_ob = json.loads(response.text)
        holidays_data = json_ob['response']['body']['items']['item']
        dataframe = json_normalize(holidays_data)
    df_ho = pd.concat([df_ho, dataframe])

In [4]:
df_ho=df_ho.reset_index()

In [5]:
df_ho['locdate']=df_ho['locdate'].reset_index(drop=True)

In [6]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [7]:
csv_to_parquet('C:/Users/gisic/gisic/jejutraffic/train.csv', 'train')
csv_to_parquet('C:/Users/gisic/gisic/jejutraffic/test.csv', 'test')

train Done.
test Done.


In [8]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [9]:
str_col = ['start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [10]:
y_train = train['target'] 

X_train = train.drop(['id', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

test = test.drop(['id', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(test.shape)

(4701217, 11)
(4701217,)
(291241, 11)


In [11]:
for i in df_ho['locdate']:
    X_train.loc[(X_train['base_date'] == i),'base_date'] = 1
X_train.loc[(X_train['base_date'] > 1),'base_date'] = 0
X_train.loc[(X_train['base_date'] == 1)]

,base_date,day_of_week,base_hour,lane_count,maximum_speed_limit,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
8,1,월,15,2,60.0,33.253074,126.506393,0,33.252183,126.506069,0
40,1,화,11,2,50.0,33.409416,126.259139,0,33.408431,126.263030,0
44,1,월,23,2,70.0,33.451731,126.366077,0,33.453585,126.353088,0
46,1,화,6,1,60.0,33.406181,126.619857,0,33.406037,126.619933,0
66,1,월,17,3,70.0,33.246996,126.524358,0,33.255106,126.540832,0
...,...,...,...,...,...,...,...,...,...,...,...
4701148,1,화,22,1,50.0,33.283593,126.720591,0,33.285638,126.720975,0
4701154,1,토,9,1,60.0,33.337244,126.695809,0,33.337398,126.696117,0
4701159,1,수,19,1,50.0,33.321552,126.244871,0,33.323197,126.248749,0
4701196,1,수,5,2,70.0,33.259229,126.593874,0,33.259086,126.593399,0


In [12]:
for i in df_ho['locdate']:
    test.loc[(test['base_date'] == i),'base_date'] = 1

In [13]:
test.loc[(test['base_date'] > 1),'base_date'] = 0

In [14]:
test.loc[(test['base_date'] == 1)]

,base_date,day_of_week,base_hour,lane_count,maximum_speed_limit,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
10,1,월,4,3,70.0,33.499973,126.516469,1,33.499848,126.519872,1
15,1,월,6,3,70.0,33.480679,126.472216,1,33.478257,126.468081,1
17,1,월,8,2,60.0,33.249949,126.505664,0,33.252183,126.506069,0
21,1,월,20,2,50.0,33.498151,126.519834,0,33.495377,126.519792,0
49,1,월,0,2,80.0,33.268749,126.394898,0,33.268781,126.394606,0
...,...,...,...,...,...,...,...,...,...,...,...
290939,1,월,21,2,60.0,33.445893,126.557958,0,33.446456,126.557373,0
290957,1,월,15,3,70.0,33.508464,126.558231,0,33.507349,126.556173,1
290980,1,월,13,2,50.0,33.246912,126.568186,0,33.247388,126.568642,0
290985,1,월,7,1,50.0,33.259995,126.491306,0,33.260056,126.490987,0


In [15]:
test

,base_date,day_of_week,base_hour,lane_count,maximum_speed_limit,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
0,0,목,17,3,70.0,33.499427,126.541298,0,33.500772,126.543837,1
1,0,화,12,2,70.0,33.258507,126.427003,0,33.258119,126.415840,0
2,0,금,2,1,60.0,33.258960,126.476508,0,33.259206,126.474687,0
3,0,목,23,3,70.0,33.473494,126.545647,0,33.471061,126.545467,0
4,0,수,17,3,70.0,33.501477,126.569223,0,33.496863,126.581230,0
...,...,...,...,...,...,...,...,...,...,...,...
291236,0,토,5,1,80.0,33.472819,126.463030,0,33.470483,126.460846,0
291237,0,금,20,2,60.0,33.305359,126.598914,0,33.300796,126.600332,0
291238,0,금,11,1,30.0,33.493624,126.496769,1,33.497500,126.496946,1
291239,0,금,7,2,60.0,33.254782,126.507014,0,33.255659,126.507333,0


In [16]:
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,0,제3교래교,33.427749,126.662335,0,52.0
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,1,KAL사거리,33.504811,126.526240,0,30.0
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,0,상창육교,33.280072,126.362147,0,61.0
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,0,서현주택,33.245565,126.566228,0,20.0
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,0,애월입구,33.462677,126.330152,0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,20211104,목,16,1,107,-,0,0,50.0,...,0,대림사거리,33.422145,126.278125,0,금덕해운,33.420955,126.273750,0,20.0
4701213,TRAIN_4701213,20220331,목,2,2,107,-,0,0,80.0,...,3,광삼교,33.472505,126.424368,0,광삼교,33.472525,126.424890,0,65.0
4701214,TRAIN_4701214,20220613,월,22,2,103,일반국도12호선,0,0,60.0,...,0,고성교차로,33.447183,126.912579,0,성산교차로,33.444121,126.912948,0,30.0
4701215,TRAIN_4701215,20211020,수,2,2,103,일반국도95호선,0,0,80.0,...,0,제6광령교,33.443596,126.431817,0,관광대학입구,33.444996,126.433332,0,73.0


In [17]:
X_train = X_train.replace('월', 0)
X_train = X_train.replace('화', 0)
X_train = X_train.replace('수', 0)
X_train = X_train.replace('목', 0)
X_train = X_train.replace('금', 0)
X_train = X_train.replace('토', 1)
X_train = X_train.replace('일', 1)
test = test.replace('월', 0)
test = test.replace('화', 0)
test = test.replace('수', 0)
test = test.replace('목', 0)
test = test.replace('금', 0)
test = test.replace('토', 1)
test = test.replace('일', 1)

In [18]:
max(X_train['day_of_week'])

1

In [19]:
X_train

,base_date,day_of_week,base_hour,lane_count,maximum_speed_limit,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
0,0,0,17,1,60.0,33.427747,126.662612,0,33.427749,126.662335,0
1,0,0,21,2,60.0,33.500730,126.529107,1,33.504811,126.526240,0
2,0,1,7,2,80.0,33.279145,126.368598,0,33.280072,126.362147,0
3,0,0,13,2,50.0,33.246081,126.567204,0,33.245565,126.566228,0
4,0,0,8,2,80.0,33.462214,126.326551,0,33.462677,126.330152,0
...,...,...,...,...,...,...,...,...,...,...,...
4701212,0,0,16,1,50.0,33.422145,126.278125,0,33.420955,126.273750,0
4701213,0,0,2,2,80.0,33.472505,126.424368,0,33.472525,126.424890,0
4701214,0,0,22,2,60.0,33.447183,126.912579,0,33.444121,126.912948,0
4701215,0,0,2,2,80.0,33.443596,126.431817,0,33.444996,126.433332,0


In [27]:
max(X_train['base_date'])

1

In [28]:
min(X_train['base_date'])

0

In [29]:
max(test['base_date'])

1

In [30]:
min(test['base_date'])

0

In [20]:
num_1=100
X_train['start_latitude'] -= 33
X_train['start_latitude'] = X_train['start_latitude']*num_1

X_train['start_longitude'] -= 126
X_train['start_longitude'] = X_train['start_longitude']*num_1

X_train['end_latitude'] -= 33
X_train['end_latitude'] = X_train['end_latitude']*num_1

X_train['end_longitude'] -= 126
X_train['end_longitude'] = X_train['end_longitude']*num_1

In [21]:
test['start_latitude'] -= 33
test['start_latitude'] = test['start_latitude']*num_1

test['start_longitude'] -= 126
test['start_longitude'] = test['start_longitude']*num_1

test['end_latitude'] -= 33
test['end_latitude'] = test['end_latitude']*num_1

test['end_longitude'] -= 126
test['end_longitude'] = test['end_longitude']*num_1

In [22]:
xg_reg = xgb.XGBRegressor(random_state=42).fit(X_train, y_train)

In [23]:
pred = xg_reg.predict(test)

In [24]:
sample_submission = pd.read_csv('C:/Users/gisic/gisic/jejutraffic/sample_submission.csv')

In [25]:
sample_submission['target'] = pred
sample_submission.to_csv("C:/Users/gisic/gisic/jejutraffic/submit.csv", index = False)

In [26]:
sample_submission

,id,target
0,TEST_000000,24.636892
1,TEST_000001,44.823383
2,TEST_000002,64.380898
3,TEST_000003,36.389881
4,TEST_000004,40.846634
...,...,...
291236,TEST_291236,46.823517
291237,TEST_291237,52.661804
291238,TEST_291238,22.385916
291239,TEST_291239,23.773336
